## gpuの使用率の調整

In [ ]:
# import tensorflow as tf

# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# session = tf.Session(config=config)

In [ ]:
%matplotlib inline

import os
import pickle
from datetime import datetime

import cv2
from PIL import Image, ImageOps
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Conv2D, Input, InputLayer, Activation, Add, UpSampling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers.normalization import BatchNormalization
from keras.models import Model, load_model
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adadelta, Adam
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

from src.preprocess import load_data
from src.merge import make_originalsize_img
from src.post_process import plot_confusion_matrix, do_plot, get_diff_data, show_diff, plot_diff_bar, plot_diff_image, make_diff_image, evaluation 
from src.models import FCN
from src.models import LKM

In [ ]:
CONTENTS = "Roads" #"Buildings" or "Roads"
TRAIN_NUM = 27700
VALID_NUM = 350
TEST_NUM = 1225

# データの取得

In [ ]:
x_train, y_train = load_data(CONTENTS, "train", TRAIN_NUM)
x_valid, y_valid = load_data(CONTENTS, "valid", VALID_NUM)
x_test, y_test = load_data(CONTENTS, "test", TEST_NUM)

# 前処理

In [ ]:
def _normalization(data, _type):
    if _type == "x":
        result = np.array([], dtype = np.float32)
        result = data.astype("float32") / 255
    elif _type == "y":
        result = np.array([], dtype = np.uint8)
        result = (data / 255).astype("uint8")
    else:
        print('please input arg( _type) "x" or "y"')
        return 0
    
    return result

In [ ]:
x_train = _normalization(x_train, "x")
y_train = _normalization(y_train, "y")
x_valid = _normalization(x_valid, "x")
y_valid = _normalization(y_valid, "y")
x_test = _normalization(x_test, "x")
y_test = _normalization(y_test, "y")

# 入出力画像の変換
- 訓練データ(x, H, W, C) -> (x, C, H, W)にする.

In [ ]:
def _reshape(data):
    if data.shape != (data.shape[0], 3, 256, 256):
        return np.transpose(data, (0, 3, 2, 1))

In [ ]:
x_train = _reshape(x_train)
y_train = _reshape(y_train)
x_valid = _reshape(x_valid)
y_valid = _reshape(y_valid)
x_test = _reshape(x_test)
y_test = _reshape(y_test)

# モデルの構築

In [ ]:
# 学習済みモデルの読込
# model = load_model("./tensorlog/01172024/learned_model.h5")

In [ ]:
fcn = FCN()
model = fcn.build()
# lkm = LKM()
# model = lkm.build()

In [ ]:
SVG(model_to_dot(model).create(prog = 'dot', format = 'svg'))

In [ ]:
model.summary()

# コンパイル

In [ ]:
model.compile(
    loss='mean_squared_error',
    optimizer = "Adam",
     metrics=['accuracy']
)

# 保存用ディレクトリ作成

In [ ]:
now = datetime.now().strftime("%m%d%H%M")
PATH = "./dataset/eva_imgs/" + now + "/"
os.makedirs(PATH + "images/")
os.makedirs(PATH + "large/")
learned_model_path = "./tensorlog/" + now + "/"
os.makedirs(learned_model_path, exist_ok=True)
test_path = './dataset/data/Roads/test/y_test/'
test_path = os.listdir(test_path)

# モデルの学習

## using fit

In [ ]:
batch_size = 8
epochs = 300
fpath = learned_model_path + '{epoch:04d}-{loss:.4f}-{val_loss:.4f}.hdf5'
es_cb = EarlyStopping(patience = 30, verbose = 0)
mc_cb = ModelCheckpoint(filepath = fpath, monitor = 'val_loss', verbose = 0, save_best_only = True, mode = 'auto',  period = 5)
tb_cb = TensorBoard(log_dir = learned_model_path, histogram_freq=0)
callbacks = []
callbacks.append(es_cb)
callbacks.append(mc_cb)
callbacks.append(tb_cb)

# datagen = ImageDataGenerator(
#     rotation_range=20,
#     vertical_flip=True,
#     horizontal_flip=True,
    
# data_format="channels_first")

# datagen.fit(x_train, rounds=3, seed=17)
# datagen.fit(y_train, rounds=3, seed=17)

# result = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size, shuffle=False, seed=17),
#                     steps_per_epoch=len(x_train) / batch_size, epochs=epochs, validation_data = (x_valid, y_valid), callbacks = callbacks)

result = model.fit(
    x = x_train,
    y = y_train,
    batch_size = batch_size,
#     initial_epoch = 90,
    epochs = epochs,
    validation_data = (x_valid, y_valid),
    verbose = 2,
    callbacks = callbacks
)

# モデルの保存

In [ ]:
model.save(learned_model_path + "learned_model.h5");

# 学習結果

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# モデルの可視化

In [ ]:
SVG(model_to_dot(model).create(prog = 'dot', format = 'svg'))
plot_model(model, to_file=learned_model_path + "model.png")

# 予測

In [ ]:
pred = model.predict(x_test, batch_size = batch_size, verbose = 1)

In [ ]:
pred = np.transpose(pred, (0, 2, 3, 1))

## 道路と背景を分離

In [ ]:
def binarization(img, NUM, size = 256):
    for i in range(size):
        for j in range(size):
            if np.sum(img[i][j] * 255) > NUM:
                img[i][j][0] = 255
                img[i][j][1] = 255
                img[i][j][2] = 255
            else:
                img[i][j][0] = 0
                img[i][j][1] = 0
                img[i][j][2] = 0
    return img

- pre_plot() 
    - 予測済みのデータを入力とする
    - input shape (1225, 256, 256, 3)

In [ ]:
NUM = 40
def pre_plot(imgs, path):
    for img, name in tqdm(zip(imgs, test_path)):
        img = binarization(img, NUM)
        img = Image.fromarray(np.uint8(img))
        # 出力画像の向きがおかしいためここで調整する
        img = img.transpose(Image.ROTATE_270)
        img = ImageOps.mirror(img)
        img.save(path + '/' + name)

In [ ]:
pre_plot(pred, PATH + "images")

# 小分けの画像を一枚に連結させる

In [ ]:
pred

In [ ]:
make_originalsize_img(PATH + "images/")

# 出力後の画像の分析

## データの整理

In [ ]:
bases, preds, filenames = get_diff_data(now)
df = show_diff(now)

## 混同行列の出力

In [ ]:
val = np.transpose(y_test, (0, 2, 3, 1))
val = (val * 255).astype(np.float32)
val = val.flatten()
pred = pred.flatten()

In [ ]:
do_plot(val, pred, learned_model_path)

# PN画像の生成

In [ ]:
validation_data = plot_diff_image(now + '/')

## 評価結果の作成(CSV)
- 以下,例  

| image_name | TruePositive | FalsePositive | TrueNegative | FalseNegative | Completeness | Correctness | Quality |
|:------- :|:---------:|:------- :|:---------:|:------- :|:---------:|:------- :|:---------:|
|10378780|19454|1541218|51789|25939|0.4286|0.2731|0.2002|
|10828720|67864|998927|540068|31541|0.6827|0.1116|0.1061|

In [ ]:
evaluation(validation_data, now)